<a href="https://colab.research.google.com/github/BeleCisterna/starter/blob/main/limpiezadatoscisterna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

No sabía dónde encontrar bases de datos, chat gpt me dio una lista de páginas donde encontrar, así pille este en kaggle, que habla sobre las reacciones/comentarios de los trailers y de videos de youtube sobre barbieheimer (o barbenhaimer).

Las bases de datos las encontré en este link https://www.kaggle.com/datasets/dnyaneshyeole/movie-trailer-reviews-oppenheimer-vs-barbie/data

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

In [27]:
oppen_df = pd.read_csv('barbieheimer.csv')
oppen_df.head(100)

,Comment,Sentiment
0,Which one ARE you watching first?<br>---------...,positive
1,Oppenheimer. THe Oppenheimer again. And probab...,neutral
2,"Oppenheimer, I will wait for Barbie on stream.",neutral
3,None I wish I went for Spider-Man and flash if...,positive
4,😂,positive
...,...,...
95,neither because too much hype,neutral
96,I watched Barbie first because I knew that Opp...,positive
97,"None, you fool.",positive
98,"neither, both shit.",positive


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_6, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_7, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(_df_8, *['Sentiment'], **{})
chart

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(_df_9, *['index', 'Sentiment'], **{'inner': 'stick'})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_10, *['index', 'count()', 'Sentiment'], **{})
chart

Le pedí que me leyera la base de datos para ver algunos comentarios, noté que habían emojis que calificaban como positivos, neutrales o negativos y no creo que haya una relación real entre el emoji y la reacción del usuario. Por eso, intenté eliminarlos.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9374 entries, 0 to 9373
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Comment    9374 non-null   object
 1   Sentiment  9369 non-null   object
dtypes: object(2)
memory usage: 146.6+ KB


In [29]:
oppen_df.head(100)

,Comment,Sentiment
0,Which one ARE you watching first?<br>---------...,positive
1,Oppenheimer. THe Oppenheimer again. And probab...,neutral
2,"Oppenheimer, I will wait for Barbie on stream.",neutral
3,None I wish I went for Spider-Man and flash if...,positive
4,😂,positive
...,...,...
95,neither because too much hype,neutral
96,I watched Barbie first because I knew that Opp...,positive
97,"None, you fool.",positive
98,"neither, both shit.",positive


Aquí le pregunté a chat gpt cómo filtrar aquello que no eran letras y me sugirió usar isalpha, pero me eliminó no solo emojis sino también números y todos los comentarios con puntuación. No me sirvió, así que lo borré y probé con el isalum. Tampoco funcionó, así que los borré para no ponerlos a correr sin querer e hice mi último intento con la recomendación de usar [a-zA-Z0-9\p{P}-] como comando. Me saltó error del /p y nunca entendí para lo que servía así que lo borré y lo puse a correr.

In [34]:
oppen_df = oppen_df[oppen_df['Comment'].apply(lambda x: re.search(r'[a-zA-Z0-9{P}-]', x) is not None if isinstance(x, str) else False)]

In [35]:
oppen_df.head(100)

,Comment,Sentiment
0,Which one ARE you watching first?<br>---------...,positive
1,Oppenheimer. THe Oppenheimer again. And probab...,neutral
2,"Oppenheimer, I will wait for Barbie on stream.",neutral
3,None I wish I went for Spider-Man and flash if...,positive
6,Pikmin 4,neutral
...,...,...
98,"neither, both shit.",positive
99,Oppenheimer,neutral
100,Strategically released to try and get people n...,positive
101,@Carlos Bingo! You right. Modern day Mocking B...,negative


Como resultado, limpié todos los emojis en la base de datos. Aunque seguramente queda por hacer y la relación Comment/Sentiment debe ser evaluada con mayor consideración.

Elegí esta base porque mis compañeros analizaron los resultados en taquilla de la película y me pareció intereante contrastar con la reacción de los trailers y videos sobre el fenómeno en lo virtual.

La base de datos puede responder:
1. La cantidad de comentarios negativos
2. La cantidad de comentarios positivos
3. La cantidad de comentarios neutrales
